In [45]:
from wit import Wit
import pandas as pd
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from bs4 import BeautifulSoup
from collections import defaultdict
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

%matplotlib inline

# 1.Selected_comments_with_witai_sentiment_results
## import comment with side effects for query

In [2]:
SE_tem_l400 = pd.read_csv("wit_ai_sentment/SE_tem_l400.csv",index_col=0,header=None)[1]
SE_tem_l400 = SE_tem_l400.iloc[3:].astype(str)
idxs = list(SE_tem_l400.index)

In [15]:
first_quest2 = pd.read_csv("wit_ai_sentment/SE_tem_l400_first_request.ccv",index_col=0)

In [16]:
print("Total results:",len(first_quest2))
first_quest2.groupby('value').size()

Total results: 30689


value
negative    16765
neutral     12010
positive     1914
dtype: int64

In [17]:
first_quest2[first_quest2['value'] == 'negative'].sample(1)['text'].iloc[0]

"I haven't noticed any major side effects"

# 2. Train the model using only these comment with side effects 

In [18]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud

from sklearn.svm import SVC

## only need to remove punctuation and stemize
stemmer = SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [19]:
## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

In [20]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0)
df = rm_sym(df)


test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0)
test = rm_sym(test)

In [21]:
train = pd.concat([df.loc[first_quest2.index], first_quest2],1)
## Generate table of words with their counts
con_vec = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
X_train_p = con_vec.fit_transform(train['review'])
#y_train = train['rating_cate']


/Users/jzz0026/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [22]:
negnornot = (train['value'] == "negative").astype(int)
negnornot.shape

(30689,)

In [39]:
X_train,X_test,y_train,y_test =  train_test_split(X_train_p,negnornot,test_size=0.25)

## Logistic regression

In [40]:
lr = LogisticRegression(penalty='l1',multi_class='auto',solver='saga')
#lr.fit(X,target_3.iloc[:,0])

cv_score = cross_val_score(lr,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
cv_score

array([0.67770103, 0.68013556, 0.67579194])

### RandomForestClassifier

In [41]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc_cv_score = cross_val_score(rfc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
rfc_cv_score

array([0.75381207, 0.75677789, 0.75609438])

In [46]:
rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc.fit(X_train,y_train)
y_test_predict = rfc.predict(X_test)
accuracy_score(y_test,y_test_predict)

0.8239280594291672

In [57]:
feature_importances = pd.DataFrame(rfc.feature_importances_,con_vec.get_feature_names())
feature_importances.sort_values(0,ascending=False)

,0
bad,0.010434
effect,0.009831
headach,0.005854
horribl,0.005587
year,0.005247
work,0.005210
day,0.004526
onli,0.004459
month,0.004272
week,0.004171


### Gradient boosting classifier

In [42]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=300)
gbc_cv_score = cross_val_score(gbc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.68943047, 0.69642857, 0.6876548 ])

### SVM

#### poly

In [43]:
svm_poly = SVC(kernel='poly')
gbc_cv_score = cross_val_score(svm_poly,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.54659195, 0.54666319, 0.54660409])

#### linear

In [44]:
svm_lin = SVC(kernel='linear')
gbc_cv_score = cross_val_score(svm_lin,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.6909944 , 0.69121481, 0.68309217])